In [1]:
%load_ext autoreload
%autoreload 2
import os
from copy import deepcopy
from os.path import join as oj

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics
from matching import *
from tqdm import tqdm

import data
import util

DIR_ORIG = '../data/celeba-hq/ims/'
DIR_GEN = oj('../data_processed/celeba-hq/generated_images_0.1')


# df contains filenames, ids, and attributes
df = data.load_all_labs()
df = df.set_index('fname_id')
df = df[['id', 'fname_final', 'age', 'gender', 'race_pred', 'race4_pred', 'Eyeglasses']] # only keep few keys to speed things up

# get fnames
fname_nps = [f for f in sorted(os.listdir(DIR_GEN)) if 'npy' in f] # these start at 00001
fnames_calculated = np.array([f[:-4] for f in fname_nps]) # these start at 00001
idxs_calculated = np.array([int(x) - 1 for x in fnames_calculated]) # this starts at 0
df = df.loc[fnames_calculated]

# load latents and calculate dists
print('reload latents...')
latents = np.array([np.load(oj(DIR_GEN, f)) for f in fname_nps])
lats = get_lat(latents)
preds = np.ones((lats.shape[0], 6)) #lats @ coefs.T + intercepts.T
weights = np.ones(preds.shape[1])
# print(ATTR_TO_INDEX)
# weights[ATTR_TO_INDEX['skin-color']] = 1e2
vecs = join_vecs(preds, lats, weights)
print('calculating gan dists...')
dists_gan = get_dists(vecs)
print('done!')

loading cached labels
reload latents...
calculating gan dists...
done!


In [2]:
# load pairwise facial dicts
print('load facial dists')
dists_facial = np.load(open('processed/13_facial_dists_pairwise.npy', 'rb')) # pkl.load()
dists_facial = dists_facial[idxs_calculated]
dists_facial = dists_facial[:, idxs_calculated]
print('facial dists loaded')

load facial dists
facial dists loaded


In [3]:
df['Race=black'] = (df['race_pred'] == 'Black').astype(int)

# find best matches

In [ ]:
'''
we want to find matched pairs
the pairs should change only one attribute at a time (but we may vary 2 to make a transect)
"main" image is matched stringently
"reference" image is matched more loosely
'''


# specify dists for matching
dists_match_name = 'facial'
if dists_match_name == 'facial':
    dists_match = dists_facial
elif dists_match_name == 'gan':
    dists_match = dists_gan
elif dists_amtch_name == 'gan_constrained':
    dists_match = dists_gan + (dists_facial > 0.6) * 1e3 # constraint for missclassificaiton

dists_ref = dists_gan
n = dists_match.shape[0]
assert df.shape[0] == n, 'df shapes must match'
assert dists_ref.shape[0] == n, 'ref shapes must match'

# specify things to vary (these should all be binary columns)
# id willl automatically be different when we vary gender, race bc these things 
# are forced to be preserved in data.py
attrs_to_vary = ['gender'] #Race=black'] # Eyeglasses, gender, 'Race=black'

# specify constraint for reference
def apply_reference_constraints(d):
    REFERENCE_CONSTRAINTS = {
        'Eyeglasses': 0, # default should be no Eyeglasses
    }
    for k in REFERENCE_CONSTRAINTS.keys():
        d = d[d[k] == REFERENCE_CONSTRAINTS[k]]
    return d

## select only ims with a reference image satisfying reference constraints
def get_idxs_satisfying_reference_constraints(df, n, dists_match, dists_ref):
    idxs_orig = pd.Series(1, df.index) #df['bool'] # np.ones(n).astype(bool)
    
    # prune anything for which dists are constant (the code for NaN)
    for i in tqdm(range(n)):
        if np.all(dists_match[i] == dists_match[i, 0]) or np.all(dists_ref[i] == dists_ref[i, 0]):
            idxs_orig.iloc[i] = False
    
    # prune things based on reference constraints
    for i in tqdm(sorted(df.id.unique())):
        d = df[(df.id == i) & idxs_orig]

        # if there is only one photo for this id, don't pick it
        if d.shape[0] == 1:
            idxs_orig.loc[d.index] = False
        
        else:
            # look for valid ref photos
            d_ref = apply_reference_constraints(d)

            # if there is no valid ref, don't pick any image with this id
            if d_ref.shape[0] < 1:
                idxs_orig.loc[d.index] = False

            # if there is 1 valid ref, don't pick the reference image
            # (we can still pick a different photo with this id)
            elif d_ref.shape[0] == 1:
                idxs_orig.loc[d_ref.index] = False 
                
    return idxs_orig.values.astype(bool)

idxs_orig = get_idxs_satisfying_reference_constraints(df, n, dists_match, dists_ref)
print('total ims', n, 'selectable ims', np.sum(idxs_orig))
print('total ids', df.id.unique().size, 'selectable ids', df[idxs_orig].id.unique().size)

In [ ]:
NUM_MATCHES = 1000
# find allowed indices for each group
subgroups = {}
for a in attrs_to_vary:
    for val in [0, 1]:
        subgroups[f'{a}_{val}'] = (df[a].values == val) & idxs_orig

matches = {}
matches_skipped = []
pairwise_constraints = np.zeros((n, n)).astype(bool) # extra constraints for matching



for match_num in tqdm(range(NUM_MATCHES)):
    # loop to create best matches
    for i, a in enumerate(attrs_to_vary):
        # this should go in order from smaller groups -> bigger groups
    #     vals = np.argsort([np.sum(subgroups[f'{a}_{val}']) for val in [0, 1]])
    #     for j, val in enumerate([0, 1]):

        ## find best match subject to constraints
        s0 = f'{a}_{0}'
        s1 = f'{a}_{1}'
        idxs0 = subgroups[s0]
        idxs1 = subgroups[s1]
        C = np.sum(idxs1)
        
        # if there are no more possible matches, stop
        if np.sum(idxs0) == 0 or np.sum(idxs1) == 0:
            NUM_MATCHES = len(matches[s0])
            break
        
        # extra constraints (e.g. previously skipped, could enforce that attributes are the same)
        dists_match_constrained = deepcopy(dists_match)
        dists_match_constrained[pairwise_constraints] = 1e5
        
        # main constraints (e.g. attributes are different, previously selected)
        dists_constrained = dists_match_constrained[idxs0][:, idxs1] # (R, C)
        
        arg = np.argmin(dists_constrained)

        # convert match arg back to space without constraints
        arg0 = arg // C
        arg1 = arg % C
        idx0 = np.where(idxs0)[0][arg0]
        idx1 = np.where(idxs1)[0][arg1]
        
        ## find reference images
        idxs_ref = pd.Series(True, df.index)
        id0 = df.iloc[idx0].id
        id1 = df.iloc[idx1].id
        idxs_ref0 = idxs_ref & (df.id == id0) & (np.arange(n) != idx0)
        idxs_ref1 = idxs_ref & (df.id == id1) & (np.arange(n) != idx1)
        assert np.sum(idxs_ref0) > 0 and np.sum(idxs_ref1) > 0, 'must have valid reference images'
        
        ## compare dists for reference images
        dists0 = dists_ref[idx0][idxs_ref0].reshape(-1, 1)
        dists1 = dists_ref[idx1][idxs_ref1].reshape(-1, 1)
        
        ## pairwise distances
        MIN_REF_DIST_THRESH_UPPER = 1e2 # 1 will weed out any crazy matches
        MIN_REF_DIST_THRESH_LOWER = 1e-2 # 1 will weed out any crazy matches
        dists_ref_dists = sklearn.metrics.pairwise_distances(dists0, dists1, metric='l1')
        C_ref = dists1.size
        arg_ref = np.argmin(dists_ref_dists)
        min_ref_dist = np.min(dists_ref_dists)
        
        # if no good reference match, then skip for now (might come back later)
        if min_ref_dist > MIN_REF_DIST_THRESH_UPPER \
        or min_ref_dist < MIN_REF_DIST_THRESH_LOWER: # match too similar (e.g. duplicate image)
            matches_skipped.append({
                'idx0': idx0,
                'idx1': idx1,
                'dist': min_ref_dist,
            })
            pairwise_constraints[idx0, idx1] = True
        else:
            ## pick best match
            arg_ref0 = arg_ref // C_ref
            arg_ref1 = arg_ref % C_ref
            idx_ref0 = np.where(idxs_ref0)[0][arg_ref0]
            idx_ref1 = np.where(idxs_ref1)[0][arg_ref1]
            
            ## save the pairs
            match = {
                s0: idx0,
                s1: idx1,
                s0 + '_ref': idx_ref0,
                s1 + '_ref': idx_ref1,
                'dist': dists_constrained[arg0, arg1],
                'dist_ref0': dists0[arg_ref0][0],
                'dist_ref1': dists1[arg_ref1][0],
            }
            for k in match.keys():
                if not k in matches:
                    matches[k] = []
                matches[k].append(match[k])

            ## remove them from further consideration
            idxs_to_remove = (df.id == id0) | (df.id == id1)
            subgroups[s0][idxs_to_remove] = False
            subgroups[s1][idxs_to_remove] = False
#         print(matches, matches_skipped)
matches = pd.DataFrame.from_dict(matches).infer_objects()
matches.to_pickle(f'processed/14_matches_{attrs_to_vary[0]}_{matches.shape[0]}_{dists_match_name}.pkl')

In [ ]:
matches = pd.read_pickle(f'processed/14_matches_{attrs_to_vary[0]}_{matches.shape[0]}')
def get_im_from_match_key(k, match):
    d = df.iloc[int(match[k])]
    return mpimg.imread(oj(DIR_ORIG, d.fname_final))

for a in attrs_to_vary:
    ks = [f'{a}_0_ref', f'{a}_0', f'{a}_1', f'{a}_1_ref']
    r = {
        k: [] for k in ks
    }
    dist_keys = {
        f'{a}_0': 'dist',
        f'{a}_1': 'dist',
        f'{a}_0_ref': 'dist_ref0',
        f'{a}_1_ref': 'dist_ref1'
    }

    for i, match in tqdm(matches.iterrows()):
        for k in r.keys():
            r[k].append(get_im_from_match_key(k, match))

    kwargs = {
        'dpi': 150
    }
    for i, k in enumerate(ks):
        util.plot_row(r[k], annot_list=matches[dist_keys[k]].round(1),
                      ylab=k.capitalize())
        plt.savefig(f'results/{k}.png', **kwargs)

In [14]:
matches

,gender_0,gender_1,gender_0_ref,gender_1_ref,dist,dist_ref0,dist_ref1
0,16503,10229,7876,11784,98.712203,113.304102,113.406888
1,1895,3902,1642,10100,99.306843,111.913963,111.655861
2,9216,9404,8076,10568,99.347743,119.696971,119.642936
3,107,10655,9778,5580,99.575084,117.101523,120.145386
4,1465,5100,3868,9320,100.023403,120.401611,121.312881
...,...,...,...,...,...,...,...
92,1370,6556,4828,612,107.610354,115.528533,116.199096
93,10848,14298,10782,13179,107.675036,93.285602,111.150179
94,185,1890,12883,4188,107.675125,124.426500,80.849539
95,8668,8208,12256,15934,107.790840,119.889391,115.886389
